In [ ]:
from google.colab import files
uploaded = files.upload()

Saving candidate_data.csv to candidate_data (3).csv


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['candidate_data.csv']))

In [ ]:
df.head()

,program_duration,difficulty_level,education,total_programs_enrolled,rating,is_pass
0,136,intermediate,Matriculation,5,1.0,0
1,131,easy,HighSchoolDiploma,2,3.0,1
2,120,easy,Matriculation,1,2.0,1
3,117,easy,Matriculation,4,1.0,1
4,131,intermediate,HighSchoolDiploma,2,4.0,1


In [ ]:
a = df[:len(df)//2]
b = df[len(df)//2 + 1 : len(df)]

In [ ]:
train_x = a.drop(columns=['is_pass'])
train_y = a.pop('is_pass')

test_x = b.drop(columns=['is_pass'])
test_y = b.pop('is_pass')

In [ ]:
CATEGORICAL_COLUMNS = ['difficulty_level','education']
NUMERIC_COLUMNS = ['program_duration','total_programs_enrolled']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = train_x[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

train_input_fn = make_input_fn(train_x, train_y)
test_input_fn = make_input_fn(test_x, test_y, num_epochs=10, shuffle=False)

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpxfqffm_8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
linear_est.train(train_input_fn)
result = linear_est.evaluate(test_input_fn)

clear_output()
print(result['accuracy'])

0.69882154


In [ ]:
result = list(linear_est.predict(test_input_fn))

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpxfqffm_8/model.ckpt-11430
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
import random
t = random.randint(0,len(train_x))
print(train_x.loc[t])
x = result[t]['probabilities'][1]
if x > 0.5:
    print("Selected")
else:
    print("Rejected")

program_duration                         134
difficulty_level                intermediate
education                  HighSchoolDiploma
total_programs_enrolled                    3
rating                                     2
Name: 31763, dtype: object
Selected
